In [107]:
import pandas as pd
import numpy as np

In [70]:
path_gc = 'GCNetwork.txt'
path_gnomad = "gnomad.v2.1.1.all_lofs.txt.bgz"

In [71]:
gnomad = pd.read_csv(path_gnomad, compression='gzip', on_bad_lines='skip', delim_whitespace=True)
gc = pd.read_csv(path_gc, skiprows = [i for i in range (1,75)], delim_whitespace=True)
gnomad.rename(columns = {'pos':'Variant'}, inplace = True)

C:\Users\Camila - bioinfo\AppData\Local\Temp\ipykernel_17304\2853696670.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  gnomad = pd.read_csv(path_gnomad, compression='gzip', on_bad_lines='skip', delim_whitespace=True)


- CFTR 
- Uma variante pode afetar mais de um transcritos, registrar quantos são afetados por cada consequencia 
- registrar quantas variantes por cada consequencia 

In [72]:
gnomad['most_severe_consequence']
#frameshift_variant
#stop_gained
#splice_acceptor_variant

0              frameshift_variant
1              frameshift_variant
2                     stop_gained
3              frameshift_variant
4              frameshift_variant
                   ...           
443764                stop_gained
443765    splice_acceptor_variant
443766                stop_gained
443767         frameshift_variant
443768                stop_gained
Name: most_severe_consequence, Length: 443769, dtype: object

In [88]:
#chrom| Variant |ref | alt | most_severe_consequence| gene_ids | gene_symbols |	transcript_ids

##transcritos por consequencias
stop = gnomad.loc[(gnomad['gene_symbols'] == 'CFTR') & (gnomad['most_severe_consequence'].str.contains('stop_gained')), 'transcript_ids']
frameshift = gnomad.loc[(gnomad['gene_symbols'] == 'CFTR') & (gnomad['most_severe_consequence'].str.contains('frameshift_variant')), 'transcript_ids']
splice = gnomad.loc[(gnomad['gene_symbols'] == 'CFTR') & (gnomad['most_severe_consequence'].str.contains('splice_acceptor_variant')), 'transcript_ids']


In [178]:
##variantes por consequencia
variant_stop_gained = gnomad.loc[(gnomad['gene_symbols'] == 'CFTR') & (gnomad['most_severe_consequence'].str.contains('stop_gained')), 'Variant'].values
variant_frameshift = gnomad.loc[(gnomad['gene_symbols'] == 'CFTR') & (gnomad['most_severe_consequence'].str.contains('frameshift_variant')), 'Variant'].values
variant_splice = gnomad.loc[(gnomad['gene_symbols'] == 'CFTR') & (gnomad['most_severe_consequence'].str.contains('splice_acceptor_variant')), 'Variant'].values

In [180]:
new_v_sg = np.pad(variant_stop_gained,(0, 6))
new_v_splice = np.pad(variant_splice,(0,40))


In [191]:
data = np.array([variant_frameshift, new_v_sg, new_v_splice])
column_name = ('frameshift variant', 'stop gained variant',  'splice acceptor variant')
pd.DataFrame(data.T, columns = column_name)

,frameshift variant,stop gained variant,splice acceptor variant
0,117120190,117120152,117170952
1,117144404,117144332,117174328
2,117149150,117149101,117174329
3,117149156,117149146,117175300
4,117149181,117170953,117175301
5,117170990,117170971,117175301
6,117171147,117170981,117180152
7,117175376,117174375,117199516
8,117175391,117175369,117199517
9,117176657,117175375,117227792
